<a href="https://colab.research.google.com/github/LBLBLBLBLBLB/machine-learning/blob/master/ML_4_TitanicClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
data =pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
data.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1, inplace=True)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [ ]:
data.isnull().any()

Survived    False
Pclass      False
Sex         False
Age          True
SibSp       False
Parch       False
Fare        False
Embarked     True
dtype: bool

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [ ]:
data['Age'] = data['Age'].fillna(data['Age'].mean())
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].value_counts().index[0])

In [ ]:
data.isnull().any()

Survived    False
Pclass      False
Sex         False
Age         False
SibSp       False
Parch       False
Fare        False
Embarked    False
dtype: bool

In [ ]:
from sklearn.preprocessing import LabelEncoder
for col in data.columns:
  if data[col].dtype == 'object':
    data[col] = LabelEncoder().fit_transform(data[col])

In [ ]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [ ]:
y = data['Survived'].values
X = data.drop('Survived', axis=1).values

In [ ]:
!pip install scikeras[tensorflow]

In [ ]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
def create_model():
 # create model
 model = Sequential()
 model.add(Dense(units=20, input_dim=X_train.shape[1], activation='relu'))
 model.add(Dense(units=20, activation='relu'))
 model.add(Dense(units=1, activation='sigmoid'))
 # Compile model
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

In [ ]:
model = KerasClassifier(model=create_model, verbose=0)

In [ ]:
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]

In [ ]:
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: 0.810440 using {'batch_size': 10, 'epochs': 100}
0.707880 (0.008121) with: {'batch_size': 10, 'epochs': 10}
0.789355 (0.014560) with: {'batch_size': 10, 'epochs': 50}
0.810440 (0.029077) with: {'batch_size': 10, 'epochs': 100}
0.705073 (0.011989) with: {'batch_size': 20, 'epochs': 10}
0.772518 (0.044513) with: {'batch_size': 20, 'epochs': 50}
0.803425 (0.027595) with: {'batch_size': 20, 'epochs': 100}
0.709274 (0.008941) with: {'batch_size': 40, 'epochs': 10}
0.721903 (0.009307) with: {'batch_size': 40, 'epochs': 50}
0.782334 (0.015436) with: {'batch_size': 40, 'epochs': 100}
0.696634 (0.008934) with: {'batch_size': 60, 'epochs': 10}
0.712070 (0.005698) with: {'batch_size': 60, 'epochs': 50}
0.786512 (0.002407) with: {'batch_size': 60, 'epochs': 100}
0.703654 (0.010429) with: {'batch_size': 80, 'epochs': 10}
0.724757 (0.019289) with: {'batch_size': 80, 'epochs': 50}
0.778115 (0.013482) with: {'batch_size': 80, 'epochs': 100}
0.664267 (0.046631) with: {'batch_size': 100, 'epochs':